# 00 — Setup & Data Audit (Walmart Sales)

**Objetivo:** validar el dataset local, realizar una EDA mínima, confirmar frecuencia semanal, definir un split temporal fijo (train/val/test) y generar `outputs/metadata.json`.

## Supuesto experimental (oracle exog)
Durante el horizonte de predicción se asume disponibilidad de **todas las variables exógenas** (`Holiday_Flag`, `Temperature`, `Fuel_Price`, `CPI`, `Unemployment`).
Este supuesto se mantiene **para todos los modelos** y debe declararse también en la memoria.

## Norma de oro (consistencia)
- Mismo split temporal y mismas métricas para todos los modelos
- Features de target (lags/rolling) construidas usando **solo pasado** (sin leakage)
- Normalización/estandarización (si aplica en un modelo): fit SOLO en train; aplicar a val/test

In [7]:
# Imports y configuración base
from __future__ import annotations

import json
import os
import platform
import sys
from pathlib import Path

import numpy as np
import pandas as pd

# Asegura imports desde src/
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import importlib
import src.common as common
importlib.reload(common)

from src.common import (
    EXOG_COLUMNS,
    REQUIRED_COLUMNS,
    DEFAULT_LAGS,
    compute_metrics,
    load_data,
    make_features,
    temporal_split,
    write_metadata,
  )

SEED = 42
np.random.seed(SEED)

DATA_PATH = PROJECT_ROOT / 'data' / 'Walmart_Sales.csv'
OUTPUTS_DIR = PROJECT_ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

print('Project root:', PROJECT_ROOT)
print('Data path:', DATA_PATH)
print('Python:', sys.version.split()[0], '| Platform:', platform.platform())

Project root: c:\Users\usuario\Documents\Master AI\TFM\MEMORIA 2.0
Data path: c:\Users\usuario\Documents\Master AI\TFM\MEMORIA 2.0\data\Walmart_Sales.csv
Python: 3.13.1 | Platform: Windows-11-10.0.26100-SP0


## 1) Carga y validación del dataset
Validaciones mínimas:
- Columnas esperadas
- Tipos básicos
- Fechas parseables
- Orden temporal por Store-Date

In [8]:
df = load_data(DATA_PATH)
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106


In [9]:
print('Shape:', df.shape)
print('Columns:', list(df.columns))
print('Date range:', df['Date'].min().date(), '->', df['Date'].max().date())
print('Stores:', df['Store'].nunique())

# Chequeos rápidos
assert set(REQUIRED_COLUMNS).issubset(df.columns)
assert df[['Store','Date']].isna().sum().sum() == 0

# Duplicados Store-Date (debería ser 0 en una serie semanal limpia)
dup = df.duplicated(['Store','Date']).sum()
print('Duplicated (Store, Date):', int(dup))

Shape: (6435, 8)
Columns: ['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
Date range: 2010-02-05 -> 2012-10-26
Stores: 45
Duplicated (Store, Date): 0


## 2) EDA mínima
Incluye:
- Missing values
- Semanas por store
- Estadísticos básicos de `Weekly_Sales`
- Señales simples de outliers (percentiles)

In [10]:
# Missing values por columna
missing = df.isna().sum().sort_values(ascending=False)
missing

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [11]:
weeks_per_store = df.groupby('Store')['Date'].nunique().sort_values()
weeks_per_store.describe()

count     45.0
mean     143.0
std        0.0
min      143.0
25%      143.0
50%      143.0
75%      143.0
max      143.0
Name: Date, dtype: float64

In [12]:
sales_desc = df['Weekly_Sales'].describe(percentiles=[0.01,0.05,0.5,0.95,0.99])
sales_desc

count    6.435000e+03
mean     1.046965e+06
std      5.643666e+05
min      2.099862e+05
1%       2.531031e+05
5%       3.084267e+05
50%      9.607460e+05
95%      2.049179e+06
99%      2.404035e+06
max      3.818686e+06
Name: Weekly_Sales, dtype: float64

## 3) Confirmación de frecuencia semanal
Chequeo: diferencia entre fechas consecutivas por tienda. En un dataset semanal, lo más común es 7 días.

In [13]:
df_sorted = df.sort_values(['Store','Date']).copy()
df_sorted['delta_days'] = df_sorted.groupby('Store')['Date'].diff().dt.days
delta_counts = df_sorted['delta_days'].value_counts(dropna=True).sort_index()
delta_counts

delta_days
7.0    6390
Name: count, dtype: int64

In [14]:
# Porcentaje de saltos semanales (7 días)
total_deltas = df_sorted['delta_days'].notna().sum()
weekly_deltas = (df_sorted['delta_days'] == 7).sum()
print('Share of 7-day deltas:', weekly_deltas / total_deltas if total_deltas else None)

Share of 7-day deltas: 1.0


## 4) Definición del split temporal (fijo)
Se define un split temporal global por fechas (mismos cortes para todas las tiendas):
- Train: desde la primera fecha hasta antes de Validation
- Validation: últimas `val_weeks` semanas antes de Test
- Test: últimas `test_weeks` semanas

Este split se reutiliza idéntico en todos los notebooks de modelos.

In [15]:
VAL_WEEKS = 8
TEST_WEEKS = 8

train_df, val_df, test_df, split_cfg = temporal_split(df, val_weeks=VAL_WEEKS, test_weeks=TEST_WEEKS)
split_cfg

NotImplementedError: iLocation based boolean indexing on an integer type is not available

In [ ]:
print('Train:', train_df['Date'].min().date(), '->', train_df['Date'].max().date(), 'rows', len(train_df))
print('Val  :', val_df['Date'].min().date(), '->', val_df['Date'].max().date(), 'rows', len(val_df))
print('Test :', test_df['Date'].min().date(), '->', test_df['Date'].max().date(), 'rows', len(test_df))

# Sanity: no overlap
assert train_df['Date'].max() < val_df['Date'].min()
assert val_df['Date'].max() < test_df['Date'].min()

## 5) Features comunes (para todos los modelos)
Se definen features comunes centralizadas:
- Lags del target: `lag_1, lag_2, lag_4, lag_8, lag_52`
- Rolling stats del target (solo pasado): `roll_mean_4, roll_mean_8, roll_std_8`
- Exógenas: `Holiday_Flag, Temperature, Fuel_Price, CPI, Unemployment`
- (Opcional) calendario: `weekofyear, month, year`

Nota: estas features se calculan en el dataframe completo pero usando únicamente información pasada (shift), por lo que **no usan el target futuro**.

In [ ]:
df_feat, feature_cols = make_features(df, lags=DEFAULT_LAGS, add_calendar=True)
print('N features:', len(feature_cols))
feature_cols

In [ ]:
# ¿Cuántas filas quedan con NaNs por lags/rolling? (esperable al inicio de cada tienda)
na_rate = df_feat[feature_cols].isna().mean().sort_values(ascending=False)
na_rate.head(10)

## 6) Metadata reproducible (outputs/metadata.json)
El archivo contiene:
- Fechas exactas de train/val/test
- Lista de features
- Semilla
- Versiones de librerías (si están instaladas)

In [ ]:
# Captura de versiones (sin internet; solo lo instalado localmente)
try:
    from importlib.metadata import version, PackageNotFoundError
except Exception:  # pragma: no cover
    version = None

def safe_version(pkg: str) -> str | None:
    if version is None:
        return None
    try:
        return version(pkg)
    except Exception:
        return None

libs = {
    'python': sys.version.split()[0],
    'numpy': safe_version('numpy'),
    'pandas': safe_version('pandas'),
    'matplotlib': safe_version('matplotlib'),
    'seaborn': safe_version('seaborn'),
    'statsmodels': safe_version('statsmodels'),
    'scikit-learn': safe_version('scikit-learn'),
}

notes = {
    'oracle_exog_assumption': 'All exogenous covariates are assumed available over the forecast horizon (oracle scenario).',
    'val_weeks': str(VAL_WEEKS),
    'test_weeks': str(TEST_WEEKS),
}

metadata_path = OUTPUTS_DIR / 'metadata.json'
write_metadata(
    metadata_path,
    split_cfg=split_cfg,
    feature_cols=feature_cols,
    seed=SEED,
    libs=libs,
    notes=notes,
)
print('Wrote:', metadata_path)

In [ ]:
# Vista rápida del metadata
print(metadata_path.read_text(encoding='utf-8')[:1200])

## 7) Checklist para los notebooks de modelos
- Reutilizar `outputs/metadata.json` para recuperar split y lista de features
- Mantener el supuesto oracle exog (explicitar)
- Evitar leakage: lags/rolling solo con pasado; escalado fit solo en train
- Guardar outputs estándar: predictions/metrics/figures